# 03 Preprocessing Steps for exp data

### 3.0 upload things
### 3.1 Notch Filtering:
50,100,150 Hz

### HP and LP filtering:
channel by channel, by the frequency bands 75 to 175 by 10Hz increments 

### FLM
group by stimuli type, and stimuli vs response

### Cat Loc
group by category


-------

#### IMPORT AND LOAD DATA

In [ ]:
# IMPORTS: 
import os
import numpy as np
import mne
import scipy
from pd_parser.parse_pd import _read_raw, _to_tsv
import pyqtgraph as pg
import pyqtgraph as plotWidget
import pandas as pd
%matplotlib inline
# %matplotlib qt


data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = ''
data_path_file = 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf'
# data_path_file = ''
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
# script_path = ''
trigger_file='PAT_3066CatLoc_triggerPD_2022-07-06.txt'
behav_file = '3066_CategoryLocalizer_9-3-2022.txt'
# trigger_channel=''


#### TRIGGER INFORMATION


In [ ]:
trig_raw_file = os.path.join(data_path, trigger_file)

with open(trig_raw_file,'r') as trig_file:
    trigs= [i.split('\n')[0].split(';') for i in trig_file.readlines()]
trigs.pop(0)
trigs = np.array(trigs,dtype=int)



#### BEHAVIORAL INFORMATION


In [ ]:
beha_raw_file = os.path.join(data_path, behav_file)
with open(beha_raw_file,'r') as beha_file:
    behav= [i.split('\n')[0] for i in beha_file.readlines()]
    print('*     Header Values are: ',behav[4])
    headers=[behav[4]].pop(0)
    [behav.pop(i) for i in [0,0,0,0,0]]
    
# BEHAVIORAL INFORMATION
df_beh = pd.DataFrame([i.split(',') for i in behav]) # 20,34,132,2,40, etc
stimNum = [i.split(',')[1] for i in behav] # 20,34,132,2,40, etc
stimName = [i.split(',')[2] for i in behav] # instrument, house, scrambled, etc.
time_repeat = [i.split(',')[3] for i in behav] # '0', '1646835641.8321767', '0', '0'
valid_trial = [i.split(',')[4] for i in behav] # 1 valid, 0 invalid -->'1', '1', '0', '1', '1', '1'
time_response = [i.split(',')[5] for i in behav] # seconds of the day '0', '0', '0', '1646835634.5921137', '0',

print(np.shape(df_beh))
# print(stimNum,stimName,time_repeat,valid_trial,time_response )
# behav = np.array(behav,dtype=int)
df_beh.head()
stimName.pop(-1)

#### Notch

In [ ]:
# from scipy import fftpack
%matplotlib qt

data_raw_file = os.path.join(data_path, data_path_file)
raw = mne.io.read_raw_edf(data_raw_file)
raw.load_data()
# raw.set_eeg_reference('average')

picks = mne.pick_channels(raw.info.ch_names,include=np.array(raw.info.ch_names)[0:-11])

## TODO: ADD COMMON AVERAGE REFERENCING BEFORE FILTERING 
# raw.plot(order=picks,n_channels=100, event_color='cyan', scalings=900, duration=100)
# raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=True)

# Notch 
print('*   NOTCH FILTERED PSD')
raw.notch_filter(np.arange(50, 200, 50), picks=picks, filter_length='10s',phase='zero')
# raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=True)
# Bandpass
print('*   BANDPASS FILTERED PSD')
# raw.filter(l_freq=0.5, h_freq=175,picks=picks,phase='zero')
raw.filter(l_freq=75, h_freq=175,picks=picks,phase='zero', filter_length=500)
# raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=True)
print('*   PLOT EEG')
# raw.load_data()
raw.plot(order=picks,n_channels=50, event_color='cyan', scalings=500, duration=100)


# raw,times= raw.get_data(picks=picks,verbose=True,start=trigs[0][0],stop=trigs[-1][1])
# trigs =trigs - trigs[0][0] 

raw.info


In [ ]:
channels = raw.ch_names[0:201]
print(len(channels))

In [12]:

# for channel in
for trial in range(0,len(trigs)):
    if trial == 0: data = pd.DataFrame(np.array(raw.get_data(picks=picks,verbose=True,start=trigs[trial][1]-500,stop=trigs[trial][1]+1500)).T,columns=channels)
    else: 
        temp = pd.DataFrame(np.array(raw.get_data(picks=picks,verbose=True,start=trigs[trial][1]-500,stop=trigs[trial][1]+1500)).T,columns=channels)
        data = data.append(temp,ignore_index=True)
        # print(np.shape(data.T))

    # for ch in range(0,np.shape(data[0])[0])
    # [plotWidget.plot(data[ch][i]) for i in range(0,len(data))]

# # raw.plot(order=picks,n_channels=25, event_color='cyan', scalings=15, duration=50)
# data = pd.DataFrame([raw.get_data(picks=picks,verbose=True,start=trigs[i][0],stop=trigs[i][1]) for i in range(0,len(trigs))],columns=stimName) #,columns=raw.ch_names
# data_resp = pd.DataFrame([raw.get_data(picks=picks,verbose=True,start=trigs[i][1],stop=trigs[i][1]+1000) for i in range(0,len(trigs))],columns=stimName) #,columns=raw.ch_names
# del raw
# data.info # trigs =trigs - trigs[0][0] 

# plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
# print(len(data))
# print(np.shape(data[0]))
# for ch in range(0,np.shape(data[0])[0])
    # [plotWidget.plot(data[ch][i]) for i in range(0,len(data))]

KeyboardInterrupt: 

In [ ]:
# data.info
# np.shape(data.FIG1)[0]/652


In [ ]:
raw.plot(order=picks,n_channels=25, event_color='cyan', scalings=15, duration=50)
data = [raw.get_data(picks=picks,verbose=True,start=trigs[i][1],stop=trigs[i][1]+1000) for i in range(0,len(trigs))]
# trigs =trigs - trigs[0][0] 

for ind in range(0,5):
    plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
    print(np.shape(data[ind][0]))
    [plotWidget.plot(data[ind][i]) for i in range(0,201)]

#### EPOCHING

In [ ]:
# plt.plot(raw.times(),raw.get_data())
%matplotlib qt

# picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='Xe1')
# fig = pg.figure(figsize=(100, 2))



channels = raw.ch_names
raw_values = raw.T
time_s = np.linspace(0,len(raw),num=len(raw))
print(np.shape(time_s),np.shape(raw))
# plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
# [plotWidget.plot(time_s,raw_values.T[i]) for i in range(0,202)]

# rem_annot = mne.Annotations(onset=trigs/,
#                             duration=[16, 11],
#                             description=['REM'] * 2)
# raw.set_annotations(rem_annot)
# (rem_events,
#  rem_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)

 
# channel_names = raw.ch_names
# pg.show()
#del raw


# events = mne.find_events(trigs)
# event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
#               'visual/right': 4, 'face': 5, 'button': 32}
# epochs = mne.Epochs(raw, events, tmin=-0.2, tmax=0.5, event_id=event_dict,
                    # preload=True)

In [ ]:
# # Some information about the DATASET: 
# print(raw.info)
# infodata = raw.info

# ## FURTHER INFORMATION
# # print(raw.info.ch_names)
# # print(raw.annotations)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# # print(picks)
# # raw.plot(order=picks, n_channels=len(picks), duration=5, scalings=1000, start=120)
# raw2 = raw.copy()
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=False,uint_cast=True, min_duration=3/raw.info['sfreq'], )
# raw2.plot(order=picks, events=events, n_channels=203, duration=20, event_color='cyan', scalings=500, start=200)
# out_dir = _TempDir()


## FILTERING

In [ ]:
# # raw2.plot(order=picks, n_channels=203, duration=20, event_color='cyan', scalings=1000, start=190)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')

# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)

# raw2.apply_hilbert(picks=picks)
# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)


In [ ]:
# FILTERING


In [ ]:

# # raw.plot(duration=60)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe')
# raw.plot(order=picks, n_channels=len(picks), duration=1600, scalings=3000, start=120)

In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='S')
# print(picks)
# raw2.plot(order=picks, n_channels=len(picks), duration=2)


In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='ST')
# print(picks)
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=True,uint_cast=True)

# raw2.plot(order=picks, n_channels=len(picks), duration=5, event_color='cyan', scalings=0.1)
# print(events)
# epochs = mne.Epochs(raw2, events=events)['2'].average().plot()

### Independend Component Analysis: 
```
[CleanData, trials, BadIdx] = reject_artefact(EventData);
save TrialIndex trials
save BadChannelIndex BadIdx

popout_ica(CleanData);

[cleanBEEPData, ~ , ~] = reject_artefact(BEEPdata);

popout_ica(cleanBEEPData);
```



In [ ]:
# # set up and fit the ICA
# ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
# ica.fit(raw)
# temporary = os.path.join(data_path, 'BadChannelIndex.mat')
# ica.exclude = scipy.io.loadmat(temporary)  # [1, 2]  # details on how we picked these are omitted here

# ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
# orig_raw = raw.copy()
# raw.load_data()
# ica.apply(raw)

# # identify channels
# temporary = os.path.join(data_path, 'Params.mat')
# params = scipy.io.loadmat(temporary) 

# # show some frontal channels to clearly illustrate the artifact removal
# chs =  params.channels # ['e1','e2']
# chan_idxs = [raw.ch_names.index(ch) for ch in chs]
# orig_raw.plot(order=chan_idxs, start=12, duration=4)
# raw.plot(order=chan_idxs, start=12, duration=4)

### Triggers: 
```
events = mne.find_events(raw, stim_channel='STI 014')
print(events[:5])  # show the first 5
```

In [ ]:
# events = mne.find_events(raw, stim_channel='STI 014')
# print(events[:5])  # show the first 5

# # event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
# #               'visual/right': 4, 'smiley': 5, 'buttonpress': 32} 
# event_dict = params.triggers

# fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw.info['sfreq'],
#                           first_samp=raw.first_samp)